# Simulated measurement file.

This file is used to simulate the measurement environment for developing purposes. In here we will have an instance of the Instrumentserver running with some dummy measurements, take a measurement, fit the data and add all of that to the backend of 🐉LabDragon🐉.

## Starting the instrumentserver.

To start the instrumentserver you need to have it installed in your environment, this is not a requirement since this is merely for testing. You can find it [here](https://github.com/toolsforexperiments/instrumentserver). Once the server is installed open a new terminal in the same directory of this file and run the command:

```bash
instrumentserver -c serverConfig.yml
```



## Imports

In [ ]:
import numpy as np

from instrumentserver.client.proxy import Client

from labcore import setup_measurements
from labcore.analysis.fit import Fit
from labcore.measurement.sweep import Sweep
from labcore.measurement import record_as, indep, dep
from labcore.analysis.mpl import fit_and_plot_1d

from labcore.data.datadict_storage import load_as_xr

from qcui_measurement.notebook import measurement_integrations
from qcui_measurement.notebook.measurement_integrations import run_and_log_measurement
from qcui_analysis.notebook import analysis
from qcui_analysis.notebook.analysis import NotebookAnalysis


## Start the client, get instruments and set parameters

In [ ]:
client = Client()

params = client.get_instrument("parameter_manager")
dummy = client.get_instrument("dummy_instrument")

DATADIR = "./data"

setup_measurements.options.instrument_clients = {"dummy": dummy}
setup_measurements.options.parameters = params
setup_measurements.DATADIR = DATADIR

measurement_integrations.SERVERADDRESS = "http://localhost:8000"
measurement_integrations.CURRENTUSER = "hoardmaster@lonelymountain.com"
measurement_integrations.BUCKETID = "09928e7c-83b3-47a0-a5bf-208d219ff18a"

analysis.SERVERADDRESS = "http://localhost:8000"
analysis.CURRENTUSER = "hoardmaster@lonelymountain.com"
analysis.BUCKETID = "09928e7c-83b3-47a0-a5bf-208d219ff18a"




## Helper functions

In [ ]:
def single_noise_point():
    return np.random.normal(0, 0.01) + 1j * np.random.normal(0, 0.01)

## Resonator

In [ ]:
if not params.has_param("resonator_response.f0"):
    params.add_parameter("resonator_response.f0", initial_value=4.94e9, unit="")

if not params.has_param("resonator_response.Q"):
    params.add_parameter("resonator_response.Q", initial_value=1000, unit="")

if not params.has_param("resonator_response.Qc"):
    params.add_parameter("resonator_response.Qc", initial_value=10000, unit="")

if not params.has_param("resonator_response.f_start"):
    params.add_parameter("resonator_response.f_start", initial_value=5e9, unit="")

if not params.has_param("resonator_response.f_end"):
    params.add_parameter("resonator_response.f_end", initial_value=5.05e9, unit="")
f0 = lambda: params.resonator_response.f0()
Q = lambda: params.resonator_response.Q()
Qc = lambda: params.resonator_response.Qc()
f_start = lambda: params.resonator_response.f_start()
f_end = lambda: params.resonator_response.f_end()


class ResonatorTransmissionResponse(Fit):
    @staticmethod
    def model(coordinates, f0, Q, Qc):
        return 1 - (Q / Qc) / (1 + 2j * Q * (coordinates - f0) / f0)

    @staticmethod
    def guess(coordinates, data):
        return dict(f0=f0(), Q=Q(), Qc=Qc())



def y_resonator(frequency):
    return 1 - (Q() / Qc()) / (1 + 2j * Q() * (frequency - f0()) / f0()) + single_noise_point()


resonator_sweep = Sweep(record_as(range(int(f_start()), int(f_end()), 10000), indep("frequency")), record_as(y_resonator, dep("S21")))

In [ ]:
data_loc = run_and_log_measurement(resonator_sweep, "resonator_response")

In [ ]:
with NotebookAnalysis(data_loc, "resonator_response_fit") as nda:

    data = load_as_xr(data_loc)

    data, result, fig = fit_and_plot_1d(ds=data, name="frequency", fit_class=ResonatorTransmissionResponse)

    nda.add(plot=fig, fitresult=result, data=data)

